# Model Define

In [20]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, LSTM, Conv1D, Lambda, Input, GlobalAveragePooling1D
from tensorflow.keras.losses import Huber
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [21]:
class Custom_Model() :
    def __init__(self, weight, input_shape) :
        self.checkpoint = weight
        self.model = self.build_model(input_shape)
        
    def build_model(self, input_shape : tuple):
        input = Input(shape=input_shape)
        x = LSTM(128, return_sequences=True, activation='tanh', dropout=0.2)(input)
        x = LSTM(64, return_sequences=True, activation='tanh', dropout=0.2)(x)
        x = LSTM(32, return_sequences=True, activation='tanh', dropout=0.2)(x)
        x = GlobalAveragePooling1D()(x)
        output = Dense(1)(x)
        return Model(input, output)
        
    
    def load_model(self) :
        self.model.load_weights(self.checkpoint)
        return self.model


In [22]:
window_size = 6
n_feature = 6

weight = './checkpoints/ckeckpointer.ckpt'
input_shape = (window_size, n_feature)

model = Custom_Model(weight, input_shape)
lstm_model = model.load_model()
lstm_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 6, 6)]            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 6, 128)            69120     
_________________________________________________________________
lstm_4 (LSTM)                (None, 6, 64)             49408     
_________________________________________________________________
lstm_5 (LSTM)                (None, 6, 32)             12416     
_________________________________________________________________
global_average_pooling1d_1 ( (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 130,977
Trainable params: 130,977
Non-trainable params: 0
_____________________________________________________

# Inference Data Preprocess

In [23]:
import pandas as pd
import os
from glob import glob
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import pyupbit

In [24]:
class Data_preprocess() :
    def __init__(self, ticker, interval, to, count) :
        self.dataset = pyupbit.get_ohlcv(ticker=ticker, interval=interval, to=to, count=count)
        self.preprocess()
        
    def preprocess(self) :
        # index(시간) 제거
        dataset_df = self.dataset.reset_index(drop=True)
        
        # value 제거
        dataset_df = dataset_df.drop(columns=['value'])
        
        # avg_price 추가
        dataset_df['avg_price'] = (dataset_df['high'] + dataset_df['low'] +
                                  dataset_df['open'] + dataset_df['close']) // 4
        
        # min max 정규화 (MinMaxScaler) 적용
        norm = MinMaxScaler()
        norm_dataset = norm.fit_transform(dataset_df)
        norm_dataset = pd.DataFrame(norm_dataset, columns=list(dataset_df.columns))
        self.data = norm_dataset
        
        # 예측될 값(label)인 10분 후 가격
        after10 = np.zeros_like(norm_dataset['close'])
        for i in range(len(norm_dataset['close']) - 1) :
            after10[i] = norm_dataset['close'][i + 1]
            self.label = after10
        
    # dataset에 window 적용
    def windowed_dataset(self, window_size, batch_size) :
        sliced_data = tf.data.Dataset.from_tensor_slices(self.data)
        sliced_data = sliced_data.window(window_size, shift=1, stride=1, drop_remainder=True)
        sliced_data = sliced_data.flat_map(lambda x : x.batch(window_size))
        
        sliced_label = tf.data.Dataset.from_tensor_slices(self.label[window_size:])
        
        sliced_dataset = tf.data.Dataset.zip((sliced_data, sliced_label))
        
        return sliced_dataset.batch(batch_size).prefetch(1)
    
        
        
        

In [25]:
ticker = 'KRW-BTC'
interval ='minute10'
to = f'2021-11-10 00:10'
count = 1000

window_size = 6
batch_size = 1

processed_data =  Data_preprocess(ticker, interval, to, count)
dataset = processed_data.windowed_dataset(window_size, batch_size)


for data in dataset.take(1):
    print("Data == ")
    print(data[0])
    
    print("\nLabel == ")
    print(data[1])

Data == 
tf.Tensor(
[[[0.22419028 0.21817058 0.23657237 0.203964   0.12238874 0.21419267]
  [0.2041498  0.20220437 0.23677737 0.20993023 0.07456903 0.20658573]
  [0.21012146 0.2092089  0.24446494 0.21266053 0.07799429 0.21255145]
  [0.21295547 0.20879687 0.24395244 0.21963798 0.0829506  0.21484395]
  [0.21973684 0.20488257 0.22539975 0.19445849 0.06683506 0.20439744]
  [0.19463563 0.1894314  0.20110701 0.17382951 0.11533618 0.18254051]]], shape=(1, 6, 6), dtype=float64)

Label == 
tf.Tensor([0.16988573], shape=(1,), dtype=float64)


In [29]:
pred = lstm_model.predict(dataset)
actual = np.asarray(processed_data.label)[6:]

pred = pred[:, 0]

print(pred.shape)
print(actual.shape)

(994,)
(994,)


# Data Parsing every 10 min
# Delete First Line and add latest data at last line  

### LIKE A QUEUE

In [62]:
# 현재 시간 문자열로 가져오기
import datetime
now = datetime.datetime.now()
str_now = now.strftime('%Y-%m-%d %H:%M')
print(type(str_now))
print(str_now)

<class 'str'>
2021-11-11 23:26


# 다음 10분을 찾아야함, 59분에서 넘어가면 날짜가 바껴야함.
# datetime은 날짜끼리 더 할 수 있음. deltatime?인가 이거 찾아서 적용해보기.

In [65]:
import datetime
today = datetime.datetime.now()
current_min = today.minute
remainder_min = current_min % 10
if remainder_min  == 0:
    current_min += 10
else :
    current_min += 10 - remainder_min 

if current_min == 60 :
    current_min = '00'
    
now = datetime.datetime.now()
coming_10m = now.strftime(f'%Y-%m-%d %H:{current_min}')
coming_10m

'2021-11-11 23:30'

In [69]:
a = 59
b = a%10
a += 10 -b 
c = a%60
c

0

In [ ]:
def parsing_10m() :
    date